# Build and run a system with a non-peptide drug

* Prepare the system splitting its components, using https://www.mdtraj.org/1.9.8.dev0/index.html
* Parameterize the drug, https://ambermd.org/tutorials/basic/tutorial4b/
* Use tleap (part of ambertools) to build
* Use miniomm to run

In [1]:
# Check availability

!which antechamber

/Users/toni/Apps/anaconda-m1/envs/openmm/bin/antechamber


In [2]:
# Split into chains. Check the one with the most modelled residues.
# In this particular case, the PDB contains 4 chains with the same sequence, all bound to NIL (nilotinib).

!head 3CS9.pdb

HEADER    TRANSFERASE                             09-APR-08   3CS9              
TITLE     HUMAN ABL KINASE IN COMPLEX WITH NILOTINIB                            
COMPND    MOL_ID: 1;                                                            
COMPND   2 MOLECULE: PROTO-ONCOGENE TYROSINE-PROTEIN KINASE ABL1;               
COMPND   3 CHAIN: A, B, C, D;                                                   
COMPND   4 FRAGMENT: KINASE DOMAIN (UNP RESIDUES 229-500);                      
COMPND   5 SYNONYM: P150, C-ABL, ABELSON MURINE LEUKEMIA VIRAL ONCOGENE HOMOLOG 
COMPND   6 1;                                                                   
COMPND   7 EC: 2.7.10.2;                                                        
COMPND   8 ENGINEERED: YES                                                      


In [3]:
import mdtraj as mdt

pdb = mdt.load("3CS9.pdb")

In [4]:
pdb.n_chains

12

In [5]:
for i,c in enumerate(pdb.topology.chains):
    print(f"Chain {i} has {c.n_residues} residues and {c.n_atoms} atoms.")
    r1 = c.residue(0)
    print(f"  First residue is {r1}")

Chain 0 has 264 residues and 2142 atoms.
  First residue is ASP233
Chain 1 has 247 residues and 2021 atoms.
  First residue is ASP233
Chain 2 has 243 residues and 1984 atoms.
  First residue is TYR232
Chain 3 has 263 residues and 2115 atoms.
  First residue is ASP233
Chain 4 has 1 residues and 39 atoms.
  First residue is NIL600
Chain 5 has 1 residues and 39 atoms.
  First residue is NIL600
Chain 6 has 1 residues and 39 atoms.
  First residue is NIL600
Chain 7 has 1 residues and 39 atoms.
  First residue is NIL600
Chain 8 has 99 residues and 99 atoms.
  First residue is HOH601
Chain 9 has 79 residues and 79 atoms.
  First residue is HOH601
Chain 10 has 63 residues and 63 atoms.
  First residue is HOH601
Chain 11 has 25 residues and 25 atoms.
  First residue is HOH601


Ok. So the best chain is chain 0 with the corresponding ligand, chain 4. Extract them.

In [6]:
pdb.atom_slice(pdb.topology.select("chainid 0")).save("chain0.pdb")

In [7]:
# Saving the NIL.pdb from mdtraj works but has wrong connectivity. Do it another way
nil=pdb.atom_slice(pdb.topology.select("chainid 4"))

In [8]:
! grep "NIL A" 3CS9.pdb > NIL.pdb

## Model the ligand

Here we use antechamber (AmberTools). See https://ambermd.org/AmberTools.php .
Several steps are involved, mostly format conversion and bond detection.

Determine the charge of the molecules.

In [9]:
! reduce NIL.pdb > NIL_r.pdb

Processing file: "NIL.pdb"
Building or keeping OH & SH Hydrogens.
VDW dot density = 16/A^2
Probe radius = 0.25A
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occupancy| < 0.66
Aromatic rings in amino acids accept hydrogen bonds.
Rotating NH3 Hydrogens.
Not processing Met methyls.
 Singles(size 2): A 600 NIL C27 : A 600 NIL C58 
 orientation 2: A 600 NIL C27 :methyl  -30: bump=-0.355, HB=0.000, total=0.304
 orientation 1: A 600 NIL C58 :methyl  150: bump=0.000, HB=0.000, total=0.000
Found 0 hydrogens (0 hets)
Standardized 0 hydrogens (0 hets)
Added 22 hydrogens (22 hets)
Adjusted 1 group(s)
If you publish work which uses reduce, please cite:
Word, et. al. (1999) J. Mol. Biol. 285, 1735-1747.
For more information see http://kinemage.biochem.duke.edu


In [10]:
# Really slow
# ! antechamber -fi pdb -i NIL_r.pdb -fo mol2 -o NIL_r.mol2 -at gaff -c bcc -pf y 
!cp NIL_r_precomputed.mol2 NIL_r.mol2

In [11]:
! parmchk2 -i NIL_r.mol2 -s gaff2 -f mol2 -o NIL.frcmod

In [12]:
!pip install git+https://github.com/giorginolab/miniomm

  Cloning https://github.com/giorginolab/miniomm to /private/var/folders/qz/7p0f8wdj4zdd8nwxm89xzhy80000gn/T/pip-req-build-w14i8hfm
  Running command git clone --filter=blob:none --quiet https://github.com/giorginolab/miniomm /private/var/folders/qz/7p0f8wdj4zdd8nwxm89xzhy80000gn/T/pip-req-build-w14i8hfm
  Resolved https://github.com/giorginolab/miniomm to commit 4517ee3b57d36abc3c355cfbc423b81c5bca24e6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [14]:
%cat leap.in

source leaprc.protein.ff19SB
source leaprc.gaff2
source leaprc.water.tip3p
# source frcmod.ionsjc_tip3p

chain0 = loadpdb chain0.pdb
check chain0

NIL = loadmol2 NIL_r.mol2
loadamberparams NIL.frcmod
check NIL

cpx = combine { chain0 NIL }

solvatebox cpx TIP3PBOX 12 iso
check cpx

# Total charge is -7

addions cpx Na+ 27 Cl- 20

# Total charge is now ~0

relax cpx
saveamberparm cpx cpx.prmtop cpx.rst7
savepdb cpx cpx.pdb

quit





In [15]:
!tleap -f leap.in

-I: Adding /Users/toni/Apps/anaconda-m1/envs/openmm/dat/leap/prep to search path.
-I: Adding /Users/toni/Apps/anaconda-m1/envs/openmm/dat/leap/lib to search path.
-I: Adding /Users/toni/Apps/anaconda-m1/envs/openmm/dat/leap/parm to search path.
-I: Adding /Users/toni/Apps/anaconda-m1/envs/openmm/dat/leap/cmd to search path.
-f: Source leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./leap.in
----- Source: /Users/toni/Apps/anaconda-m1/envs/openmm/dat/leap/cmd/leaprc.protein.ff19SB
----- Source of /Users/toni/Apps/anaconda-m1/envs/openmm/dat/leap/cmd/leaprc.protein.ff19SB done
Log file: ./leap.log
Loading parameters: /Users/toni/Apps/anaconda-m1/envs/openmm/dat/leap/parm/parm19.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA + ff19SB
Loading parameters: /Users/toni/Apps/anaconda-m1/envs/openmm/dat/leap/parm/frcmod.ff19SB
Reading force field modification type file (frcmod)
Reading title:
ff19SB AA-specific backbone CMAPs for protein 07/25/2019
Loa